## 🔹 1. **Q-Learning (Vanilla Q-Learning)**

Q-learning is a **model-free reinforcement learning algorithm** that learns the value of taking an action in a given state.
It uses a **Q-table** (state-action value table) to store values.

**Update Rule**:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \Big[ r + \gamma \max_{a'} Q(s', a') - Q(s,a) \Big]
$$

* $s$: current state
* $a$: action taken
* $r$: reward
* $s'$: next state
* $\alpha$: learning rate
* $\gamma$: discount factor

👉 Works well in **small discrete state spaces**, but struggles with large or continuous spaces since the Q-table becomes huge.

**Example:**
Suppose an agent in a grid world wants to reach a goal.

* States = grid cells
* Actions = {up, down, left, right}
* The Q-table might look like:

| State | Up  | Down | Left | Right |
| ----- | --- | ---- | ---- | ----- |
| (0,0) | 0   | 0.2  | 0    | 0.1   |
| (0,1) | 0.5 | 0.1  | 0.3  | 0.4   |

The agent updates this table until it learns the best path.

In [1]:
# Q-Learning with State Discretization (Beginner Friendly Example)
# -------------------------------------------------------------
# This example uses Q-learning on the CartPole environment.
# Since the environment has continuous states, we discretize them
# into bins (categories) to apply tabular Q-learning.

import gymnasium as gym
import numpy as np

# -------------------------------------------------------------
# 1. Create the environment
# -------------------------------------------------------------
# "CartPole-v1" is a classic control problem.
# Goal: Keep the pole balanced by moving left or right.

env = gym.make("CartPole-v1")

# -------------------------------------------------------------
# 2. Discretization Setup
# -------------------------------------------------------------
# The CartPole observation has 4 continuous values:
# [cart position, cart velocity, pole angle, pole angular velocity]
# We'll discretize only the pole angle and pole angular velocity,
# since they are most important for balancing.

n_bins = (6, 12)  # 6 bins for angle, 12 bins for angular velocity

# Define limits for clipping (CartPole docs give infinite ranges for velocity)
obs_space = np.array([
    [-4.8, 4.8],       # cart position (unused)
    [-5.0, 5.0],       # cart velocity (unused)
    [-0.418, 0.418],   # pole angle (approx -24° to +24°)
    [-5.0, 5.0]        # pole angular velocity
])

def discretize(obs):
    """
    Convert continuous observation values (angle & angular velocity)
    into discrete bin indices.
    """
    # Select only indices 2 (angle) and 3 (angular velocity)
    ratios = []
    for i, bins in zip([2, 3], n_bins):
        low, high = obs_space[i]
        ratio = (obs[i] - low) / (high - low)
        new_obs = int(round((bins - 1) * ratio))
        # Clip to stay inside bin range
        new_obs = np.clip(new_obs, 0, bins - 1)
        ratios.append(new_obs)
    return tuple(ratios)

# -------------------------------------------------------------
# 3. Initialize Q-table
# -------------------------------------------------------------
# Q-table shape: (angle_bins, velocity_bins, actions)

Q = np.zeros(n_bins + (env.action_space.n,))

# Hyperparameters
alpha = 0.1   # learning rate
gamma = 0.99  # discount factor
eps = 1.0     # exploration rate (epsilon for epsilon-greedy)

# -------------------------------------------------------------
# 4. Training Loop
# -------------------------------------------------------------
# We'll run multiple episodes, update Q-values using Q-learning,
# and slowly reduce epsilon (exploration) over time.

n_episodes = 2000

for episode in range(n_episodes):
    obs, _ = env.reset()
    state = discretize(obs)
    done = False
    total_reward = 0

    while not done:
        # Choose action (epsilon-greedy)
        if np.random.rand() < eps:
            action = env.action_space.sample()  # explore
        else:
            action = np.argmax(Q[state])        # exploit best action

        # Take action in the environment
        next_obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_state = discretize(next_obs)

        # Q-learning update rule
        best_next = np.max(Q[next_state])
        Q[state + (action,)] += alpha * (reward + gamma * best_next - Q[state + (action,)])

        # Move to next state
        state = next_state
        total_reward += reward

    # Decay epsilon (exploration rate)
    eps = max(0.01, eps * 0.995)

    # Print progress every 100 episodes
    if episode % 100 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")

# -------------------------------------------------------------
# 5. Summary
# -------------------------------------------------------------
# - This is a **tabular Q-learning** example.
# - It works because we discretized the continuous state space.
# - Later, we will compare this with **Deep Q-Learning**, which
#   uses a neural network to handle continuous states directly.

Episode 0, Total Reward: 22.0
Episode 100, Total Reward: 18.0
Episode 200, Total Reward: 55.0
Episode 300, Total Reward: 196.0
Episode 400, Total Reward: 234.0
Episode 500, Total Reward: 154.0
Episode 600, Total Reward: 196.0
Episode 700, Total Reward: 11.0
Episode 800, Total Reward: 216.0
Episode 900, Total Reward: 166.0
Episode 1000, Total Reward: 179.0
Episode 1100, Total Reward: 145.0
Episode 1200, Total Reward: 160.0
Episode 1300, Total Reward: 164.0
Episode 1400, Total Reward: 140.0
Episode 1500, Total Reward: 180.0
Episode 1600, Total Reward: 146.0
Episode 1700, Total Reward: 219.0
Episode 1800, Total Reward: 170.0
Episode 1900, Total Reward: 245.0


## 🔹 2. **Deep Q-Learning (DQN or D-Q Learning)**

Instead of a Q-table, we use a **neural network** to approximate the Q-function:

$$
Q(s,a;\theta) \approx Q(s,a)
$$

* $\theta$: parameters of the neural network
* Input: state (can be high-dimensional, e.g., images)
* Output: Q-values for each possible action

### Key Features of DQN

1. **Experience Replay**: Store past experiences $(s,a,r,s')$ in a replay buffer, and sample mini-batches to break correlation between consecutive updates.
2. **Target Network**: Maintain a separate network for stable Q-value updates.

**Update Rule (with NN):**

$$
L(\theta) = \Big[ r + \gamma \max_{a'} Q(s',a';\theta^-) - Q(s,a;\theta) \Big]^2
$$

where $\theta^-$ are the parameters of the target network.

---

### Example: Playing Atari (Breakout 🎮)

* **Q-Learning**: Not feasible (huge state space, every pixel arrangement is a state).
* **DQN**: Input the raw image into a convolutional neural network → output Q-values for {move left, move right, fire}.

  * Example: The NN might learn that in state (ball near paddle, moving right), the Q-value for action “move right” is highest.

---

## 🔑 Key Differences

| Feature              | Q-Learning                    | Deep Q-Learning (DQN)                            |        |   |   |                     |
| -------------------- | ----------------------------- | ------------------------------------------------ | ------ | - | - | ------------------- |
| Value Representation | **Q-table** (explicit lookup) | **Neural Network** (function approximation)      |        |   |   |                     |
| State Space          | Small, discrete               | Large/continuous, high-dimensional               |        |   |   |                     |
| Memory               | Needs table of size (         | S                                                | \times | A | ) | Needs weights of NN |
| Stability            | More stable, but limited      | Needs tricks (experience replay, target network) |        |   |   |                     |
| Applications         | Gridworld, simple games       | Atari, robotics, real-world tasks                |        |   |   |                     |

---

✅ **In short:**

* **Q-learning** = Good for small toy problems.
* **DQN (Deep Q-learning)** = Scales Q-learning using neural nets → can solve complex problems like playing video games or controlling robots.

In [2]:
# Deep Q-Learning with Neural Network (Beginner Friendly Example)
# -------------------------------------------------------------
# In this example, we solve the CartPole environment using
# Deep Q-Learning (DQN). Instead of discretizing states,
# we use a neural network to approximate Q-values directly.

import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# -------------------------------------------------------------
# 1. Create the environment
# -------------------------------------------------------------
# Same CartPole environment as before.

env = gym.make("CartPole-v1")

# -------------------------------------------------------------
# 2. Define the Neural Network (Q-value Approximator)
# -------------------------------------------------------------
# Input: state (4 continuous values)
# Output: Q-value for each action (left or right)

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.out(x)

# -------------------------------------------------------------
# 3. Setup Policy & Target Networks
# -------------------------------------------------------------
# Policy net: Learns during training
# Target net: Provides stable targets (updated less often)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

policy_net = DQN(state_dim, action_dim)
target_net = DQN(state_dim, action_dim)
target_net.load_state_dict(policy_net.state_dict())  # sync initially

optimizer = optim.Adam(policy_net.parameters(), lr=1e-3)

# -------------------------------------------------------------
# 4. Replay Buffer (Experience Replay)
# -------------------------------------------------------------
# Stores past experiences and samples random batches
# to break correlation between consecutive steps.

replay_buffer = deque(maxlen=10000)
batch_size = 64
gamma = 0.99  # discount factor
eps = 1.0    # exploration rate (epsilon-greedy)

# -------------------------------------------------------------
# 5. Action Selection (Epsilon-Greedy)
# -------------------------------------------------------------
def select_action(state):
    if random.random() < eps:
        return env.action_space.sample()  # explore
    state = torch.FloatTensor(state).unsqueeze(0)
    return policy_net(state).argmax().item()  # exploit

# -------------------------------------------------------------
# 6. Training Loop
# -------------------------------------------------------------
# Each episode: play the game, store experiences, update network.

n_episodes = 500

for episode in range(n_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Choose action
        action = select_action(state)

        # Step in the environment
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Save transition in replay buffer
        replay_buffer.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        # Train only if buffer has enough samples
        if len(replay_buffer) > batch_size:
            batch = random.sample(replay_buffer, batch_size)
            s, a, r, ns, d = zip(*batch)

            s = torch.FloatTensor(s)
            a = torch.LongTensor(a).unsqueeze(1)
            r = torch.FloatTensor(r).unsqueeze(1)
            ns = torch.FloatTensor(ns)
            d = torch.FloatTensor(d).unsqueeze(1)

            # Current Q(s,a)
            q_values = policy_net(s).gather(1, a)

            # Target Q-values: r + gamma * max_a' Q(s',a')
            max_next_q = target_net(ns).max(1, keepdim=True)[0]
            target = r + gamma * max_next_q * (1 - d)

            # Loss (MSE)
            loss = nn.MSELoss()(q_values, target.detach())

            # Gradient descent
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # ---------------------------------------------------------
    # 7. Exploration Decay & Target Network Update
    # ---------------------------------------------------------
    eps = max(0.01, eps * 0.995)  # slowly reduce exploration

    if episode % 10 == 0:
        target_net.load_state_dict(policy_net.state_dict())

    # Print progress every 50 episodes
    if episode % 50 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")

# -------------------------------------------------------------
# 8. Summary
# -------------------------------------------------------------
# - Unlike Q-learning with discretization, here we use a neural
#   network to handle the continuous state space.
# - The target network and replay buffer are key innovations
#   that make Deep Q-Learning stable.
# - This will be compared with the tabular Q-learning example.


Episode 0, Total Reward: 14.0


C:\Users\mohdf\AppData\Local\Temp\ipykernel_14524\188651980.py:105: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  s = torch.FloatTensor(s)


Episode 50, Total Reward: 88.0
Episode 100, Total Reward: 88.0
Episode 150, Total Reward: 93.0
Episode 200, Total Reward: 101.0
Episode 250, Total Reward: 28.0
Episode 300, Total Reward: 35.0
Episode 350, Total Reward: 107.0
Episode 400, Total Reward: 114.0
Episode 450, Total Reward: 91.0
